In [1]:
### ----- GERAÇÃO/CARREGAMENTO DO CLASSIFICADOR (futuras iterações) ----- ###

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict
import torch
import os
import pandas as pd

# Caminho para salvar ou carregar o modelo
MODEL_DIR = "/kaggle/working/modelo_bert_skills/"

# === 1. Carregando os dados
df = pd.read_csv("/kaggle/input/500k-atualizados/sample_skills_atualizado_500k.csv")
df["label"] = df["label"].astype(int)

# === 2. Limpeza dos dados
df = df[df["skill_name"].notnull()]  # remove nulos
df["skill_name"] = df["skill_name"].astype(str)  # garante que tudo é string

# === 3. Divisão treino/teste
df_amostra = df.sample(frac=1, random_state=42).reset_index(drop=True)
df_treino = df_amostra.iloc[:150000]
tam = len(df)
df_teste = df_amostra.iloc[150001:tam]

# === 4. Conversão para HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_treino),
    "test": Dataset.from_pandas(df_teste)
})

# === 5. Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# === 6. Pré-processamento
def preprocess(exemplos):
    return tokenizer(
        exemplos["skill_name"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

# Tokenizando sem remover nenhuma coluna ainda
dataset = dataset.map(preprocess, batched=True)

# Renomeando label
dataset = dataset.rename_column("label", "labels")

# Definindo colunas para o modelo, mas preservando as extras
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
    output_all_columns=True  # <- mantém skill_id e skill_name
)


# Data collator necessário para batching correto com padding dinâmico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# === 7. Carregamento ou criação do modelo
if os.path.exists(MODEL_DIR) and (
    "pytorch_model.bin" in os.listdir(MODEL_DIR) or "model.safetensors" in os.listdir(MODEL_DIR)):
    print("Carregando modelo salvo...")
    model_bert = BertForSequenceClassification.from_pretrained(MODEL_DIR)
    tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
    print("Modelo carregado...")
else:
    print("Treinando modelo BERT do zero...")

    model_bert = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased", num_labels=3
    )

    # Detecta se há GPU disponível
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Dispositivo usado:", device)

    # Aumenta o batch size para acelerar (se a memória permitir)
    args = TrainingArguments(
        output_dir=MODEL_DIR,
        num_train_epochs=4,  # diminui para teste mais rápido (pode aumentar depois)
        per_device_train_batch_size=16,  # <== Aumentado para maior eficiência
        per_device_eval_batch_size=32,
        warmup_steps=500,
        weight_decay=0.01,
        save_strategy="epoch",
        save_total_limit=1,  # só o mais recente
        logging_dir="/kaggle/working/logs",
        logging_steps=100,
        report_to=[],  # desativa W&B
        fp16=torch.cuda.is_available(),  # ativa FP16 se houver GPU com suporte
    )

    trainer = Trainer(
        model=model_bert,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    print("Iniciando treinamento...")
    trainer.train()
    print("Treinamento finalizado.")

    model_bert.save_pretrained(MODEL_DIR)
    tokenizer.save_pretrained(MODEL_DIR)
    print("Modelo salvo em:", MODEL_DIR)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

Carregando modelo salvo...
Modelo carregado...


In [4]:
!du -sh /kaggle/working/*

36K	/kaggle/working/clusters_nivel0.joblib
992K	/kaggle/working/clusters_nivel_max0.joblib
428K	/kaggle/working/hdbscan_nivel0.pkl
13M	/kaggle/working/hdbscan_nivel_max0.pkl
2.7G	/kaggle/working/modelo_bert_skills
84K	/kaggle/working/reduzido_nivel0.joblib
2.5M	/kaggle/working/reduzido_nivel_max0.joblib
4.0K	/kaggle/working/state.db
27M	/kaggle/working/umap_nivel0.pkl
813M	/kaggle/working/umap_nivel_max0.pkl
186M	/kaggle/working/vetores_embeddings.npy
1.3M	/kaggle/working/visualizacao1_clusters.json
23M	/kaggle/working/visualizacao_clusters_aux.json
23M	/kaggle/working/visualizacao_clusters_final.json
1.3M	/kaggle/working/visualizacao_clusters.json


In [12]:
### ----- GERAÇÃO DOS VETORES DE EMBEDDINGS ----- ###

import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Carregar base
caminho_csv = "/kaggle/input/500k-atualizados/sample_skills_atualizado_500k.csv"
df = pd.read_csv(caminho_csv)

skills_descricao = []
skills_info = []

# Armazenar apenas as skills (removendo lixo)
for i in range(0,tam,1):
    skill_original = df.iloc[i+1, 1]
    if df.iloc[i+1, 2] != 2:
        skills_descricao.append(skill_original)
        skills_info.append(df.iloc[i+1, 2])

# 2. Embeddings (carregando modelo)
modelo_nome = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(modelo_nome)
modelo = AutoModel.from_pretrained(modelo_nome)

def gerar_embeddings(sentencas):
    entradas = tokenizer(sentencas, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        saida = modelo(**entradas)
    return torch.mean(saida.last_hidden_state, dim=1).cpu().numpy()

# 3. Processamento em batches
tamanho_batch = 512  
embeddings_lista = []

for inicio in range(0, len(skills_descricao), tamanho_batch):
    fim = min(inicio + tamanho_batch, len(skills_descricao))
    batch = skills_descricao[inicio:fim]
    vetores_batch = gerar_embeddings(batch)
    embeddings_lista.append(vetores_batch)
    print(f'Processado: {fim}/{len(skills_descricao)}')

# Junta tudo em um só array
vetores_embeddings = np.vstack(embeddings_lista)

# Salva
np.save('/kaggle/working/vetores_embeddings.npy', vetores_embeddings)

Processado: 512/126914
Processado: 1024/126914
Processado: 1536/126914
Processado: 2048/126914
Processado: 2560/126914
Processado: 3072/126914
Processado: 3584/126914
Processado: 4096/126914
Processado: 4608/126914
Processado: 5120/126914
Processado: 5632/126914
Processado: 6144/126914
Processado: 6656/126914
Processado: 7168/126914
Processado: 7680/126914
Processado: 8192/126914
Processado: 8704/126914
Processado: 9216/126914
Processado: 9728/126914
Processado: 10240/126914
Processado: 10752/126914
Processado: 11264/126914
Processado: 11776/126914
Processado: 12288/126914
Processado: 12800/126914
Processado: 13312/126914
Processado: 13824/126914
Processado: 14336/126914
Processado: 14848/126914
Processado: 15360/126914
Processado: 15872/126914
Processado: 16384/126914
Processado: 16896/126914
Processado: 17408/126914
Processado: 17920/126914
Processado: 18432/126914
Processado: 18944/126914
Processado: 19456/126914
Processado: 19968/126914
Processado: 20480/126914
Processado: 20992/12

In [ ]:
### ---- GERAÇÃO DOS CLUSTERS NIVEL 0 (ESPECIFICO) ---- ###

import os
import joblib
import numpy as np
import pandas as pd
import umap
import hdbscan

# Defina o valor de nivel (exemplo: i = 0)
nivel = 0

# Checa se todos os arquivos necessários existem
umap_path = f"/kaggle/working/umap_nivel_max{i}.pkl"
hdbscan_path = f"/kaggle/working/hdbscan_nivel_max{i}.pkl"
reduzido_path = f"/kaggle/working/reduzido_nivel_max{i}.joblib"
clusters_path = f"/kaggle/working/clusters_nivel_max{i}.joblib"

if all(os.path.exists(path) for path in [umap_path, hdbscan_path, reduzido_path, clusters_path]):
    print("Arquivos encontrados! Carregando modelos e dados já processados...")
    umap_modelo = joblib.load(umap_path)
    modelo_cluster = joblib.load(hdbscan_path)
    reduzido = joblib.load(reduzido_path)
    clusters = joblib.load(clusters_path)
else:
    print("Arquivos não encontrados. Processando do zero...")
    # Carrega os embeddings já calculados
    vetores_embeddings = np.load('/kaggle/working/vetores_embeddings.npy')

    n_neighbors = 50          # Quantidade de vizinhos considerados pelo UMAP (valores altos deixam clusters mais globais/espalhados)
    min_dist = 1.0            # Distância mínima entre pontos no espaço reduzido do UMAP (1.0 deixa os clusters bem separados, sem aglomeração)
    min_cluster_size = 5      # Tamanho mínimo para um cluster ser considerado pelo HDBSCAN (clusters menores que isso são tratados como ruído)
    n_components = 5          # n_components = 5 para clusterização

    # Inicialização dos Modelos UMAP e HDBSCAN
    umap_modelo = umap.UMAP(
        n_neighbors=n_neighbors,
        n_components=n_components,
        min_dist=min_dist,
        metric='cosine',
        random_state=42
    )

    reduzido = umap_modelo.fit_transform(vetores_embeddings)

    modelo_cluster = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=2,
        metric='euclidean'
    )

    clusters = modelo_cluster.fit_predict(reduzido)

    # Salva os modelos em arquivo
    joblib.dump(umap_modelo, umap_path)
    joblib.dump(modelo_cluster, hdbscan_path)
    joblib.dump(reduzido, reduzido_path)
    joblib.dump(clusters, clusters_path)

# Continue normalmente a partir daqui...
df_final = pd.DataFrame({"skill": skills_descricao})
df_final["x"] = reduzido[:, 0]
df_final["y"] = reduzido[:, 1]
df_final[f"cluster_n0"] = clusters

# Nomeação com GROQ LLaMA3
nomes = {}
cont = 0
for c in sorted(np.unique(clusters)):
    if c == -1:
        nomes[c] = "Ruído"
        continue
    amostras = df_final[df_final[f"cluster_n0"] == c]["skill"].tolist()[:13]
    nomes[c] = nomear_cluster(amostras, nivel)
    cont += 1
    print(f"{cont}/{len(np.unique(clusters))}")
    if(cont == 100):
        df_final[f"nome_n0"] = df_final[f"cluster_n0"].map(nomes)
        # Junção
        df_json_aux = df_final.to_dict(orient="records")
        with open("/kaggle/working/visualizacao_clusters_aux.json", "w", encoding="utf-8") as f:
            json.dump(df_json_aux, f, ensure_ascii=False, indent=2)
                

df_final[f"nome_n0"] = df_final[f"cluster_n0"].map(nomes)

print("Clusterização Nivel 0 Concluída")

In [24]:
print(nomes)

{-1: 'Ruído', 0: 'Excelência em processamento de dados.', 1: 'Domínio Pacote Office', 2: 'Domínio extenso Pacote Office.', 3: 'Excel Expertise Data Analysis Mastery', 4: 'Domínio Pacote Office Multifacetado.', 5: 'Domínio Pacote Office Amplo.', 6: 'Excelência em processamento de dados.', 7: 'Excelência em Processamento de Dados.', 8: 'Organização estrutural dados.', 9: 'Organiza categoriza classifica estrutura', 10: 'Trabalho em equipe colaborativo eficaz.', 11: 'Excelência em processamento de dados.', 12: 'Excelência em processamento de dados.', 13: 'Pacote Office Domínio Profundo', 14: 'Excelência em processamento de dados.', 15: 'Excelência em Processamento de Dados.', 16: 'Organização estruturada dados classificação.', 17: 'Organização estrutural dados classificacão.', 18: 'Organização estrutural dados complexos.', 19: 'Organiza categoriza classifica estruturas dados.', 20: 'Organização estrutural dados classificados.', 21: 'Domínio pacote Office', 22: 'Organização estrutural dados

In [31]:
clusters_unicos = sorted(np.unique(clusters))
inicio_iteracao = 4124
cont = inicio_iteracao - 1  # já passou essas iterações
for c in clusters_unicos[inicio_iteracao - 1:]:
    if c == -1:
        nomes[c] = "Ruído"
        continue
    
    amostras = df_final[df_final[f"cluster_n0"] == c]["skill"].tolist()[:13]
    nomes[c] = nomear_cluster(amostras, nivel)
    
    cont += 1
    print(f"{cont}/{len(clusters_unicos)}")
    
    if cont % 100 == 0:
        df_final[f"nome_n0"] = df_final[f"cluster_n0"].map(nomes)
        df_json_aux = df_final.to_dict(orient="records")
        with open("/kaggle/working/visualizacao_clusters_aux.json", "w", encoding="utf-8") as f:
            json.dump(df_json_aux, f, ensure_ascii=False, indent=2)

# Salvar resultado final
df_final[f"nome_n0"] = df_final[f"cluster_n0"].map(nomes)
df_json_final = df_final.to_dict(orient="records")
with open("/kaggle/working/visualizacao_clusters_final.json", "w", encoding="utf-8") as f:
    json.dump(df_json_final, f, ensure_ascii=False, indent=2)

print("Clusterização Nível 0 Concluída a partir da iteração", inicio_iteracao)

Gestão Redes Sociais Profissional
4124/5588
Análise Técnica Comunicação
4125/5588
Redes Sociais Profissionalmente Dominadas
4126/5588
Redes Conhecimento Técnico Avanç
4127/5588
Habilidades Sociais Comunicativas
4128/5588
Habilidades de Classificação Avançadas
4129/5588
Conhecimento Informática Intermediário Geral
4130/5588
Desenvolvimento Profissional Online
4131/5588
Colaboração Eficiência Qualidade Equipe
4132/5588
Informática Técnica Comunicação Efetiva
4133/5588
Redes Configuração Gerenciamento Avançado
4134/5588
Gestão Redes Sociais
4135/5588
Análise Desenvolvimento Software
4136/5588
Redes Administracao Ferramentas Sociais
4137/5588
Gestão Redes Sociais Avançadas
4138/5588
Gestão Informação Comunicação Saúde
4139/5588
Análise Dados Software Especializados
4140/5588
Visão Integrada Processos Aprendizagem
4141/5588
Segurança Higiene Técnicas Avançadas
4142/5588
Trabalho Autônomo Proativo Eficaz
4143/5588
Redes Computadores Protocolos Segurança.
4144/5588
Trabalho Independente Proat

In [23]:
df_final[f"nome_n0"] = df_final[f"cluster_n0"].map(nomes)
# Junção
df_json_aux = df_final.to_dict(orient="records")
with open("/kaggle/working/visualizacao_clusters_aux.json", "w", encoding="utf-8") as f:
    json.dump(df_json_aux, f, ensure_ascii=False, indent=2)

In [32]:
# ---- GERAÇÃO DOS CLUSTERS NIVEL 1 (INTERMEDIARIO) ----

from sklearn.cluster import AgglomerativeClustering
import json
import pandas as pd

# Carregue o JSON em uma lista de dicionários
with open("/kaggle/working/visualizacao_clusters.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

# Converta para DataFrame
df_final = pd.DataFrame(dados)

# Calcule centroides e nomes (ordem garantida)
centroides = []
nomes_n0 = []
clusters_n0_validos = []
for c in sorted(df_final['cluster_n0'].unique()):
    if c == -1:
        continue
    pontos = df_final[df_final['cluster_n0'] == c]
    centroides.append([pontos['x'].mean(), pontos['y'].mean()])
    nomes_n0.append(pontos['nome_n0'].iloc[0])
    clusters_n0_validos.append(c)
centroides = np.array(centroides)

# Agglomerative clustering
n_clusters_nivel1 = 150
cluster_model = AgglomerativeClustering(n_clusters=n_clusters_nivel1, linkage='ward')
labels_nivel1 = cluster_model.fit_predict(centroides)

# Nomeação dos superclusters
nomes_nivel1 = {}
for super_c in sorted(np.unique(labels_nivel1)):
    clusters_filhos = [i for i, l in enumerate(labels_nivel1) if l == super_c]
    nomes_filhos = [nomes_n0[i] for i in clusters_filhos]
    nome_super = nomear_cluster(nomes_filhos, nivel=1)
    nomes_nivel1[super_c] = nome_super

# Mapeando cluster_n0 para o supercluster_n1
cluster_n0_to_n1 = {c: labels_nivel1[i] for i, c in enumerate(clusters_n0_validos)}
df_final['cluster_n1'] = df_final['cluster_n0'].map(cluster_n0_to_n1)
df_final.loc[df_final['cluster_n0'] == -1, 'cluster_n1'] = -1

df_final['nome_n1'] = df_final['cluster_n1'].map(nomes_nivel1)
df_final.loc[df_final['cluster_n1'] == -1, 'nome_n1'] = "Ruído"

print("Clusterização Nivel 1 Concluída")

Proatividade Foco Implementação
Gestão de Riscos
Vendas Liderança Online
Análise Dados TI
Desenvolvimento Software Linux
Marketing Digital Tecnologia
Gestão Industrial Eficiência
Desenvolvimento Web Backend
Gestão Liderança Comunicação
Marketing Digital Automacao
Gestão Processos Análise
Análise Dados
Análise Dados Otimização
Gestão e Organização
Desenvolvimento Web Fullstack
Gestão Qualidade Liderança
Gerenciamento Liderança Eficiência
Gestão de Conhecimento
Análise Contabilidade Legislação
Banco Dados Liderança
Comunicação Colaboração Excelência
Comunicação Empática Adaptável
Desenvolvedor Full Stack
Atendimento Foco Resultados
Gestão Eficiência Excelência
TI Marketing Digital
Gestão Tecnologia Design
Desenvolvimento Web Frontend
Gestão Equipe Comunicação
Análise Inovação Taxonomia
Comunicação Liderança Adaptabilidade
Fiscal Contabilidade Gestão
Gerenciamento Taxonomia Tecnologia
Gestão Taxonomia Inova
Análise Social Ética
Análise Desenvolvimento Web
Análise Estratégica Foco
Análise 

In [33]:
# ---- GERAÇÃO DOS CLUSTERS NIVEL 2 (ABRANGENTE) ----

from sklearn.cluster import AgglomerativeClustering

# Calcule centroides e nomes dos clusters_n1 (com verificação)
centroides_n1 = []
nomes_n1 = []
clusters_n1_validos = []
for c in sorted(df_final['cluster_n1'].unique()):
    if c == -1:
        continue
    pontos = df_final[df_final['cluster_n1'] == c]
    if len(pontos) == 0:
        continue
    centroides_n1.append([pontos['x'].mean(), pontos['y'].mean()])
    nomes_n1.append(pontos['nome_n1'].iloc[0])
    clusters_n1_validos.append(c)
centroides_n1 = np.array(centroides_n1)

# Agglomerative clustering para o nível 2
n_clusters_nivel2 = 30  # ajuste conforme necessário
cluster_model2 = AgglomerativeClustering(n_clusters=n_clusters_nivel2, linkage='ward')
labels_nivel2 = cluster_model2.fit_predict(centroides_n1)

# Nomeação dos superclusters do nível 2
nomes_nivel2 = {}
for super_c in sorted(np.unique(labels_nivel2)):
    clusters_filhos = [i for i, l in enumerate(labels_nivel2) if l == super_c]
    nomes_filhos = [nomes_n1[i] for i in clusters_filhos]
    nome_super = nomear_cluster(nomes_filhos, nivel=2)
    nomes_nivel2[super_c] = nome_super

# Mapeando cluster_n1 para o supercluster_n2
cluster_n1_to_n2 = {c: labels_nivel2[i] for i, c in enumerate(clusters_n1_validos)}
df_final['cluster_n2'] = df_final['cluster_n1'].map(cluster_n1_to_n2)
df_final.loc[df_final['cluster_n1'] == -1, 'cluster_n2'] = -1

df_final['nome_n2'] = df_final['cluster_n2'].map(nomes_nivel2)
df_final.loc[df_final['cluster_n2'] == -1, 'nome_n2'] = "Ruído"

print("Clusterização Nivel 2 Concluída")

Gestão Corporativa
Gestão Organizacional
Desenvolvimento Tecnologia
Análise Liderança
Gestão Comunicação
Tecnologia Informação
Desenvolvimento Tecnologia
Gestão Financeira
Desenvolvimento Tecnologia
Ciência Dados
Gestão Pessoas
Desenvolvimento Tecnologia
Gestão Tecnologia
Gestão Empresarial
Habilidades Sociais
Análise Dados
Gestão Corporativa
Comunicação Eficaz
Gestão Empresarial
Trabalho Equipe
Comunicação Liderança
Gestão Industrial
Ciência Dados
Marketing Digital
Análise Tecnológica
Gestão Taxonomia
Categoria Gestão
Segurança Veículo
Gestão Cultural
Desenvolvimento Tecnológico
Clusterização Nivel 2 Concluída


In [37]:
### ---- GERAÇÃO FINAL DO JSON PARA VISUALIZACAO VIA WEB JAVASCRIPT ---- ###

import numpy as np
import pandas as pd
import torch
import umap
import hdbscan
import joblib
import json
from transformers import AutoTokenizer, AutoModel
from groq import Groq

# Junção
df_json = df_final.to_dict(orient="records")
with open("/kaggle/working/visualizacao1_clusters.json", "w", encoding="utf-8") as f:
    json.dump(df_json, f, ensure_ascii=False, indent=2)

print("Pipeline com 4 níveis concluído. Arquivo JSON pronto para visualização web.")

Pipeline com 4 níveis concluído. Arquivo JSON pronto para visualização web.


In [2]:
### ---- FUNCAO PARA NOMEAÇÃO DOS CLUSTERS (3 NIVEIS) ---- ###

### ---- llama3-8b-8192 / gemma2-9b-it / llama3-8b-8192 ---- ###
!pip install groq
import numpy as np
import pandas as pd
import torch
import umap
import hdbscan
import joblib
import json
from transformers import AutoTokenizer, AutoModel
from groq import Groq
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("client")


client = Groq(api_key=secret_value_0)

def nomear_cluster(skills_cluster, nivel):
    niveis_info = {
        0: {"palavras": 4, "exemplo": "Técnicas Avançadas Processamento Linguagem"},
        1: {"palavras": 3, "exemplo": "Desenvolvimento Web Frontend"},
        2: {"palavras": 2, "exemplo": "Engenharia Software"}
    }
    info = niveis_info[nivel]
    limite = info["palavras"]
    exemplo = info["exemplo"]

    msg = ", ".join(skills_cluster)

    user_msg = (
        f"Você apenas tem direito de falar {limite} palavras (sem uso de virgulas, numeros, pontos finais, outros pontos"
        f"asteriscos) resuma essas skills ({msg}) e retorne algo semelhante a isso: {exemplo}."
    )

    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": f"Você é um profissional que irá me ajudar a classificar grupos"},
            {"role": "user", "content": user_msg}
        ],
        temperature=0.6,
        max_tokens=12,
        top_p=1,
        stream=False,
    )

    try:
        resposta = completion.choices[0].message.content.strip()
        print(resposta)
        # Remove frases comuns caso escapem
        for padrao in [
            "Aqui está", "Aqui estão", "Claro", "Nomes para grupos", "Nomes para Grupos",
            "Nomes curtos", "Exemplo", ":", "#", "-", "Habilidades"
        ]:
            if resposta.lower().startswith(padrao.lower()):
                resposta = resposta[len(padrao):].strip()
        return resposta
    except Exception as e:
        print(f"Erro ao processar resposta: {e}")
        return "Unknown"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.3 MB/s eta 0:00:00


2025-06-23 17:12:01.025004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750698721.207588      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750698721.263732      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import plotly.graph_objects as go

def gerar_hover_text(linha):
    return f"""
    <b>{linha['skill']}</b><br>
    Nível 0: {linha['nome_n0']} (ID: {linha['cluster_n0']})<br>
    """

df_final["hover_text"] = df_final.apply(gerar_hover_text, axis=1)

text=df_final["hover_text"],
hoverinfo="text",

fig = go.Figure()

fig.add_trace(go.Scattergl(
    x=df_final["x"],
    y=df_final["y"],
    mode="markers",
    marker=dict(
        color=df_final["cluster_n3"],  # cor por cluster mais específico
        colorscale="Viridis",
        size=6,
        opacity=0.8,
        showscale=True,
        colorbar=dict(title="Cluster Nível 3")
    ),
    text=df_final["hover_text"],
    hoverinfo="text",
    name="Skills"
))

fig.update_layout(
    title="📌 Mapa Interativo de Skills com Hierarquia Completa",
    xaxis=dict(showgrid=True, visible=True),
    yaxis=dict(showgrid=True, visible=True),
    height=750
)

fig.show()